In [1]:
import pandas as pd
import numpy as np

pd.options.plotting.backend = "plotly"

In [2]:
hourly_data_path = 'students_drahi_production_consumption_hourly.csv'
hourly_data = pd.read_csv(hourly_data_path)

hourly_data

,datetime,AirTemp,pres,rain,rh,wd,ws,Global_Solar_Flux,Diffuse_Solar_Flux,Direct_Solar_Flux,...,kw_heatingcoolingtotal_zone1,kw_heatingcoolingtotal_zone2,kw_lights_zone1,kw_lights_zone2,kw_total_zone1,kw_total_zone2,kw_ventilation_zone1,kw_ventilation_zone2,kw_water_heater_zone2,plugs_zone2
0,2022-01-01 00:00:00+00:00,10.279,1005.058,0.000,95.870,187.576,1.758,-0.271,-0.370,-0.110,...,-1.798,-2.037,-0.002,-0.222,2.091,2.308,-0.027,-0.030,-0.004,-0.156
1,2022-01-01 01:00:00+00:00,10.318,1004.790,0.000,95.335,218.280,2.401,-0.371,-0.499,-0.231,...,-1.794,-2.170,-0.002,-0.278,2.086,2.504,-0.027,-0.044,-0.004,-0.157
2,2022-01-01 02:00:00+00:00,9.901,1004.791,0.013,95.699,185.351,1.821,-0.846,-0.999,-0.235,...,-1.795,-2.202,-0.002,-0.238,2.085,2.501,-0.026,-0.043,-0.004,-0.157
3,2022-01-01 03:00:00+00:00,9.444,1004.554,0.000,95.881,193.537,1.760,-0.632,-0.755,-0.173,...,-1.811,-2.238,-0.002,-0.240,2.100,2.539,-0.026,-0.044,-0.004,-0.157
4,2022-01-01 04:00:00+00:00,8.587,1004.524,0.000,96.738,174.806,1.613,-0.395,-0.489,-0.110,...,-1.814,-2.287,-0.002,-0.280,2.102,2.952,-0.027,-0.044,-0.356,-0.156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2023-12-31 19:00:00+00:00,7.523,983.610,0.000,77.079,248.157,5.871,-1.114,-1.283,-0.246,...,-1.058,-1.896,-0.130,-0.031,1.659,2.058,-0.031,-0.022,0.000,-0.206
17516,2023-12-31 20:00:00+00:00,7.021,984.488,0.000,81.753,246.199,5.223,-1.130,-1.270,0.178,...,-1.070,-1.886,-0.121,-0.019,1.661,2.038,-0.031,-0.023,0.000,-0.206
17517,2023-12-31 21:00:00+00:00,6.746,984.980,0.000,83.942,245.813,4.572,-0.944,-1.041,0.281,...,-7.507,-1.890,-0.120,-0.019,8.045,2.049,-0.031,-0.023,0.000,-0.215
17518,2023-12-31 22:00:00+00:00,6.703,985.161,0.000,83.873,241.623,4.568,-0.935,-1.029,0.203,...,-3.758,-1.868,-0.120,-0.019,4.361,2.021,-0.031,-0.023,0.000,-0.207


In [8]:
hourly_data_index = hourly_data.set_index("datetime")

hourly_data_index

,AirTemp,pres,rain,rh,wd,ws,Global_Solar_Flux,Diffuse_Solar_Flux,Direct_Solar_Flux,Downwelling_IR_Flux,...,kw_heatingcoolingtotal_zone1,kw_heatingcoolingtotal_zone2,kw_lights_zone1,kw_lights_zone2,kw_total_zone1,kw_total_zone2,kw_ventilation_zone1,kw_ventilation_zone2,kw_water_heater_zone2,plugs_zone2
datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,10.279,1005.058,0.000,95.870,187.576,1.758,-0.271,-0.370,-0.110,342.922,...,-1.798,-2.037,-0.002,-0.222,2.091,2.308,-0.027,-0.030,-0.004,-0.156
2022-01-01 01:00:00+00:00,10.318,1004.790,0.000,95.335,218.280,2.401,-0.371,-0.499,-0.231,328.399,...,-1.794,-2.170,-0.002,-0.278,2.086,2.504,-0.027,-0.044,-0.004,-0.157
2022-01-01 02:00:00+00:00,9.901,1004.791,0.013,95.699,185.351,1.821,-0.846,-0.999,-0.235,288.659,...,-1.795,-2.202,-0.002,-0.238,2.085,2.501,-0.026,-0.043,-0.004,-0.157
2022-01-01 03:00:00+00:00,9.444,1004.554,0.000,95.881,193.537,1.760,-0.632,-0.755,-0.173,309.317,...,-1.811,-2.238,-0.002,-0.240,2.100,2.539,-0.026,-0.044,-0.004,-0.157
2022-01-01 04:00:00+00:00,8.587,1004.524,0.000,96.738,174.806,1.613,-0.395,-0.489,-0.110,323.557,...,-1.814,-2.287,-0.002,-0.280,2.102,2.952,-0.027,-0.044,-0.356,-0.156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00+00:00,7.523,983.610,0.000,77.079,248.157,5.871,-1.114,-1.283,-0.246,259.846,...,-1.058,-1.896,-0.130,-0.031,1.659,2.058,-0.031,-0.022,0.000,-0.206
2023-12-31 20:00:00+00:00,7.021,984.488,0.000,81.753,246.199,5.223,-1.130,-1.270,0.178,261.117,...,-1.070,-1.886,-0.121,-0.019,1.661,2.038,-0.031,-0.023,0.000,-0.206
2023-12-31 21:00:00+00:00,6.746,984.980,0.000,83.942,245.813,4.572,-0.944,-1.041,0.281,289.698,...,-7.507,-1.890,-0.120,-0.019,8.045,2.049,-0.031,-0.023,0.000,-0.215


In [ ]:
from sklearn.preprocessing import FunctionTransformer

#Little hack to "freeze" transformers, since the data in the input is flattened 

def fit_and_freeze(transformer):
    fitted = [False]

    def func(x):
        if not fitted[0]:
            transformer.fit(x)
            fitted[0] = True
        return transformer.transform(x)

    return FunctionTransformer(func)

In [9]:
time = pd.to_datetime(hourly_data['datetime']).values.astype('datetime64[s]')
power_consumption = hourly_data['kw_total_zone2'].values

dec = [] # daily energy consumption
t_dec = []

for ti, t in enumerate(time):
    tmp_t = pd.Timestamp(t)

    if np.isclose(tmp_t.hour, 0) and np.isclose(tmp_t.minute, 0):

        day_end = np.datetime64(tmp_t + pd.Timedelta(days=1))
        ind = np.where((time > tmp_t) & (time < day_end), True, False)

        if len(time[ind]) > 0 and not np.isnan(power_consumption[ind]).any():
            t_dec.append(np.datetime64(tmp_t).astype('datetime64[s]'))
            dec.append(np.trapz(power_consumption[ind], time[ind].astype(int))/3600)

t_dec = np.array(t_dec)
dec = np.array(dec)

In [4]:
N = 7 # N days of predictors beforehand
final_ind = []
final_hourly = []

predictor_window = pd.Timedelta(days=N)

for ti, t in enumerate(t_dec):
    tmp_t = pd.Timestamp(t)

    ind = np.where((time >= tmp_t - predictor_window) & (time < tmp_t), True, False) # finding indices within the N prior days

    bad_ind = np.isnan(hourly_data.iloc[ind, 1::].values)
    if len(time[ind]) >= 24 * N and not bad_ind.any(): # rejecting any data with NaNs; useful for the student dataset
        final_ind.append(ti)
        final_hourly.append(hourly_data.iloc[ind, 1::].values)

target_time = t_dec[final_ind]
predictors = np.array(final_hourly)

X = pd.DataFrame(predictors.reshape(len(predictors), -1))
y = pd.DataFrame(np.array(dec[final_ind]))

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=148)


preprocessor = Pipeline(steps=[])

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', Ridge(alpha=1e8))
])

clf.fit(X, y)

#reg = Ridge(alpha=1e8)
#reg.fit(x_train, y_train)
#y_pred = reg.predict(x_test)

In [14]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,4526,4527,4528,4529,4530,4531,4532,4533,4534,4535
312,18.428,995.407,0.0,57.516,77.123,3.501,-1.257,-1.533,-0.360,319.558,...,-0.219,-0.258,-0.199,-0.021,1.308,0.467,-0.332,0.024,-0.000,-0.171
198,15.011,998.739,0.0,84.504,143.207,1.827,-0.990,-1.102,-0.348,315.333,...,-0.725,-0.425,-0.005,-0.026,1.309,0.948,-0.022,0.007,-0.005,-0.549
7,4.383,994.138,0.0,79.744,335.499,4.431,-0.134,-0.225,-0.071,310.776,...,-2.372,-1.864,-0.001,-0.055,2.662,2.163,-0.024,0.011,-0.004,-0.343
425,4.341,980.571,0.0,82.835,194.219,1.599,-0.499,-0.541,0.103,305.489,...,-7.752,-2.072,-0.007,-0.019,10.125,2.376,-2.042,0.027,-0.001,-0.418
409,8.994,980.793,0.0,92.324,244.227,3.599,-0.941,-1.343,0.475,274.657,...,-1.274,-0.950,-0.196,-0.053,3.340,1.340,-1.493,0.025,0.000,-0.354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,11.973,1003.873,0.0,95.989,260.565,2.727,-0.329,-0.390,-0.177,368.473,...,-0.198,-1.974,-0.004,-0.064,0.786,2.291,-0.022,0.008,-0.005,-0.395
210,8.981,980.178,0.0,93.478,236.877,3.002,-1.109,-1.249,-0.415,269.589,...,-1.293,-1.865,-0.200,-0.026,2.075,2.238,-0.022,0.008,-0.005,-0.502
225,2.029,985.750,0.0,90.118,19.370,2.041,-0.461,-0.481,-0.103,287.682,...,-1.864,-1.971,-0.004,-0.076,2.263,2.493,-0.022,0.009,-0.042,-0.583
180,10.227,1003.284,0.0,68.071,108.445,2.003,-1.110,-1.293,-0.440,272.149,...,-0.803,-0.721,-0.006,-0.027,1.335,1.082,-0.022,-0.045,-0.005,-0.365
